In [19]:
# Importing dependencies 
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Importing time traveled to work (county and state) csvs and concatonating into a sigle dataframe
path1 = 'project-1/Resources/Travel Time to Work'
files1 = os.listdir(path1)
travel_df = pd.DataFrame()
for file in files1:
    temp1 = pd.read_csv(path1 + '\\' + file, skiprows=[1])
    temp1 = temp1[['B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', \
        'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'NAME']]
    #Renaming columns 
    temp1.columns = ['Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
        'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
        'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min', 'State']
    # Adding a column for respective years
    temp1['Year'] = file.split(' ')[0]
    # reordering columns
    temp1 = temp1[['Year', 'State', 'Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
        'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
        'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min']]
    # Removing Puerto Rico
    temp1 = temp1[temp1['State'].str.contains('Puerto Rico')==False]
    # Appending each new temporary dataframe into one
    travel_df = travel_df.append(temp1)

# Importing time traveled to work (county and state) csvs and concatonating into a sigle dataframe
path2 = 'project-1/Resources/Travel Time County and State Level'
files2 = os.listdir(path2)
cs_travel_df = pd.DataFrame()
for file in files2:
    temp = pd.read_csv(path2 + '\\' + file, skiprows=[1])
    temp = temp[['B08131_001E', 'B08131_002E', 'B08131_003E', 'B08131_004E', 'B08131_005E', 'NAME']]
    # Renaming columns 
    temp.columns = ['Estimate Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
        'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
        'Worked Outside State of Residence Estimate Aggregate (min)', 'State']
    # Adding a column for respective years
    temp['Year'] = file.split(' ')[0]
    # Reordering columns
    temp = temp[['Year', 'State', 'Estimate Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
        'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
        'Worked Outside State of Residence Estimate Aggregate (min)']]
    # Removing '-' values 
    temp['Worked Outside County of Residence Estimate Aggregate (min)'] = pd.to_numeric(temp['Worked Outside County of Residence Estimate Aggregate (min)'], \
                                                                                       errors='coerce')
    temp = temp[temp['Worked Outside County of Residence Estimate Aggregate (min)'] != '-']
    # Removing Puerto Rico
    temp = temp[temp['State'].str.contains('Puerto Rico')==False]
    # Appending each new temporary dataframe into one
    cs_travel_df = cs_travel_df.append(temp)

# Importing HPI csv and making into a dataframe
hpi_quarterly = pd.read_csv('project-1/Resources/HPI/HPI Quarterly State Level.csv')
# Removing Warning column
hpi_quarterly = hpi_quarterly[['state', 'yr', 'qtr', 'index_nsa', 'index_sa']]
# Renaming columns
hpi_quarterly.columns = ['State', 'Year', 'Quarter', 'NSA Index', 'SA Index']
# Gropuby to get mean of SA and NSA index for each state and year
hpi_yearly = hpi_quarterly.groupby(['State', 'Year'], as_index=False).agg({'NSA Index': 'mean', 'SA Index': 'mean'})
# Renaming columns again 
hpi_yearly.columns = ['State', 'Year', 'Yearly Average Index (NSA)', 'Yearly Average Index (SA)']
# Only keeping years 2010-2019
hpi_yearly = hpi_yearly.loc[(hpi_yearly['Year'].astype(int) >= 2010) & (hpi_yearly['Year'].astype(int) <= 2019)]

In [12]:
us_state_to_abbrev = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", \
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", \
    "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", \
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", \
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", \
    "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", \
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", \
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC"
}

In [13]:
# Changing state names to abbreviations
travel_df = travel_df.replace({'State': us_state_to_abbrev})
cs_travel_df = cs_travel_df.replace({'State': us_state_to_abbrev})

# Merging HPI and travel time dataframes
travels = pd.merge(travel_df, cs_travel_df, on=['State', 'Year'], how='left')
# Changing HPI year column back into strings to maintain compatibility 
hpi_yearly['Year'] = hpi_yearly['Year'].astype(str)
travel_hpi = pd.merge(travels, hpi_yearly, on=['State', 'Year'], how='left')

# Saving merged dataframe in Resources folder
#travel_hpi.to_csv('Resources/2010 to 2019 HPI and Travel Time to Work.csv')

In [14]:
travel_hpi.head()

,Year,State,Total Estimate,Estimate Less than 5 min,Estimate 5 to 9 min,Estimate 10 to 14 min,Estimate 15 to 19 min,Estimate 20 to 24 min,Estimate 25 to 29 min,Estimate 30 to 34 min,...,Estimate 45 to 59 min,Estimate 60 to 89 min,Estimate 90 or more min,Estimate Aggregate Travel Time (min),Worked in State of Residence Estimate Aggregate (min),Worked in County of Residence Estimate Aggregate (min),Worked Outside County of Residence Estimate Aggregate (min),Worked Outside State of Residence Estimate Aggregate (min),Yearly Average Index (NSA),Yearly Average Index (SA)
0,2010,AL,1895987,59051,189676,274724,330572,301885,117283,264594,...,135707,76275,36428,45734230,42487215,25986650,16500565.0,3247015,180.7875,180.1300
1,2010,AK,324619,30199,56593,58333,59252,44082,16062,24726,...,12229,8194,5665,6100595,6069265,4973180,1096085.0,31325,219.9000,218.8725
2,2010,AZ,2470950,66596,243712,346279,385741,376165,166696,390622,...,179809,99719,39654,60533900,58957890,53765435,5192455.0,1576015,183.1500,182.7550
3,2010,AR,1189709,60584,171728,203607,207225,163084,73975,136006,...,60070,35293,19736,25178550,23519785,14298840,9220950.0,1658765,178.3125,178.0825
4,2010,CA,15097170,346580,1331209,2102009,2350056,2255417,851250,2224075,...,1188689,1019773,452499,405422285,402553330,285220410,117332920.0,2868955,164.7850,163.9675


# HPI Summary

In [33]:
df_2010_2019 = travel_hpi.loc[(travel_hpi['Year'] == '2010') | (travel_hpi['Year'] == '2019')]
state_groupby = df_2010_2019.groupby('State')
# Calculating change in HPI (both NSA and SA) over time (2010 to 2019) for each state
states = df_2010_2019['State'].unique()

change_nsa = state_groupby['Yearly Average Index (NSA)'].diff().dropna() 
change_sa = state_groupby['Yearly Average Index (SA)'].diff().dropna()
pct_change_nsa = (state_groupby['Yearly Average Index (NSA)'].pct_change().dropna())*100
pct_change_sa = (state_groupby['Yearly Average Index (SA)'].pct_change().dropna())*100
hpi_change = pd.DataFrame({'State': states, 'Change in NSA Index': change_nsa, '% Change in NSA Index': pct_change_nsa, \
    'Change in SA Index': change_sa, '% Change in SA Index': pct_change_sa})
hpi_change = hpi_change.reset_index(drop=True)
hpi_change.style.format({'% Change in NSA Index': '{:,.2f}%', '% Change in SA Index': '{:,.2f}%'})

,State,Change in NSA Index,% Change in NSA Index,Change in SA Index,% Change in SA Index
0,AL,48.627500,26.90%,48.467500,26.91%
1,AK,45.180000,20.55%,45.012500,20.57%
2,AZ,151.690000,82.82%,151.385000,82.83%
3,AR,46.727500,26.21%,46.552500,26.14%
4,CA,124.192500,75.37%,123.555000,75.35%
5,CO,216.760000,84.16%,215.672500,84.15%
6,CT,7.127500,4.24%,7.125000,4.25%
7,DE,24.967500,13.36%,25.005000,13.43%
8,DC,232.167500,73.43%,230.285000,73.41%
9,FL,139.995000,78.74%,139.717500,78.74%


In [35]:
# Min and Max percentages
max_nsa = hpi_change['% Change in NSA Index'].max()
min_nsa = hpi_change['% Change in NSA Index'].min()
max_sa = hpi_change['% Change in SA Index'].max()
min_sa = hpi_change['% Change in SA Index'].min()
# States corresponding to min and max
state_max_nsa = hpi_change[hpi_change['% Change in NSA Index'] == max_nsa]['State'].item()
state_min_nsa = hpi_change[hpi_change['% Change in NSA Index'] == min_nsa]['State'].item() 
state_max_sa = hpi_change[hpi_change['% Change in SA Index'] == max_sa]['State'].item()
state_min_sa = hpi_change[hpi_change['% Change in SA Index'] == min_sa]['State'].item()

print(f'Highest % change working in county of residence: {round(max_nsa,2)}%, {state_max_nsa}')
print(f'Lowest % change working in county of residence: {round(min_nsa,2)}%, {state_min_nsa}')
print(f'Highest % change working in state of residence: {round(max_sa,2)}%, {state_max_sa}')
print(f'Lowest % change working in state of residence: {round(min_sa,2)}%, {state_min_sa}')

Highest % change working in county of residence: 110.02%, NV
Lowest % change working in county of residence: 4.24%, CT
Highest % change working in state of residence: 110.05%, NV
Lowest % change working in state of residence: 4.25%, CT


# Time Traveled to Work Summary

In [22]:
# Calculating change in travel time over time (2010 to 2019) for each state
change_less5 = state_groupby['Estimate Less than 5 min'].pct_change().dropna() * 100 
change_5to9 = state_groupby['Estimate 5 to 9 min'].pct_change().dropna() * 100 
change_10to14 = state_groupby['Estimate 10 to 14 min'].pct_change().dropna() * 100
change_15to19 = state_groupby['Estimate 15 to 19 min'].pct_change().dropna() * 100
change_20to24 = state_groupby['Estimate 20 to 24 min'].pct_change().dropna() * 100
change_10to14 = state_groupby['Estimate 10 to 14 min'].pct_change().dropna() * 100
change_25to29 = state_groupby['Estimate 25 to 29 min'].pct_change().dropna() * 100
change_30to34 = state_groupby['Estimate 30 to 34 min'].pct_change().dropna() * 100
change_35to39 = state_groupby['Estimate 35 to 39 min'].pct_change().dropna() * 100
change_40to44 = state_groupby['Estimate 40 to 44 min'].pct_change().dropna() * 100
change_45to59 = state_groupby['Estimate 45 to 59 min'].pct_change().dropna() * 100
change_60to89 = state_groupby['Estimate 60 to 89 min'].pct_change().dropna() * 100
change_90more = state_groupby['Estimate 90 or more min'].pct_change().dropna() * 100

travel_time_change = pd.DataFrame({'State': states, '% Change <5 min': change_less5, '% Change 5 to 9 min': change_5to9, \
    '% Change 10 to 14 min': change_10to14, '% Change 15 to 19 min': change_15to19, '% Change 20 to 24 min': change_20to24, \
    '% Change 25 to 29 min': change_25to29, '% Change 30 to 34 min': change_30to34, '% Change 35 to 39 min': change_35to39, \
    '% Change 40 to 44 min': change_40to44, '% Change 45 to 59 min': change_45to59, '% Change 60 to 89 min': change_60to89, \
    '% Change > 89 min': change_90more})
travel_time_change = travel_time_change.reset_index(drop=True)
travel_time_change.style.format({'% Change <5 min': '{:,.2f}%', '% Change 5 to 9 min': '{:,.2f}%', \
    '% Change 10 to 14 min': '{:,.2f}%', '% Change 15 to 19 min': '{:,.2f}%', '% Change 20 to 24 min': '{:,.2f}%', \
    '% Change 25 to 29 min': '{:,.2f}%', '% Change 30 to 34 min': '{:,.2f}%', '% Change 35 to 39 min': '{:,.2f}%', \
    '% Change 40 to 44 min': '{:,.2f}%', '% Change 45 to 59 min': '{:,.2f}%', '% Change 60 to 89 min': '{:,.2f}%', \
    '% Change > 89 min': '{:,.2f}%'})


,State,% Change <5 min,% Change 5 to 9 min,% Change 10 to 14 min,% Change 15 to 19 min,% Change 20 to 24 min,% Change 25 to 29 min,% Change 30 to 34 min,% Change 35 to 39 min,% Change 40 to 44 min,% Change 45 to 59 min,% Change 60 to 89 min,% Change > 89 min
0,AL,-9.02%,3.86%,4.14%,6.20%,5.53%,13.54%,5.94%,17.58%,17.98%,22.75%,31.86%,24.03%
1,AK,-12.57%,6.94%,5.39%,5.69%,0.71%,4.63%,8.99%,-40.85%,-13.51%,-3.73%,11.51%,41.13%
2,AZ,8.80%,16.26%,7.84%,14.51%,19.70%,38.89%,14.31%,41.42%,20.75%,43.17%,75.40%,74.95%
3,AR,-0.55%,-0.18%,9.76%,4.93%,3.82%,-2.89%,8.17%,15.40%,19.07%,20.93%,19.44%,17.67%
4,CA,-13.34%,-7.24%,-2.21%,6.26%,7.93%,26.68%,17.75%,33.56%,34.68%,36.60%,54.71%,79.29%
5,CO,-20.38%,-2.60%,6.75%,16.05%,21.17%,23.06%,28.28%,51.88%,43.59%,40.59%,63.14%,41.90%
6,CT,-18.06%,-6.77%,-0.37%,-1.74%,3.00%,7.35%,-4.22%,25.71%,20.75%,10.78%,25.26%,50.15%
7,DE,-0.33%,-2.16%,11.61%,1.63%,7.54%,20.15%,20.14%,30.07%,27.96%,22.28%,36.40%,60.44%
8,DC,-17.71%,19.95%,10.56%,8.72%,0.67%,21.24%,24.11%,70.83%,75.64%,69.32%,45.98%,3.04%
9,FL,-3.27%,3.13%,2.35%,10.48%,13.50%,23.44%,28.27%,37.08%,56.50%,46.76%,76.96%,65.38%


In [26]:
#Calculating change in worked in state and change in worked in county (both NSA and SA) over time (2010 to 2019) for each state
change_in_county = state_groupby['Worked in County of Residence Estimate Aggregate (min)'].diff().dropna()
change_in_state = state_groupby['Worked in State of Residence Estimate Aggregate (min)'].diff().dropna()
change_out_county = state_groupby['Worked Outside County of Residence Estimate Aggregate (min)'].diff().dropna()
change_out_state = state_groupby['Worked Outside State of Residence Estimate Aggregate (min)'].diff().dropna()

pct_change_in_county = (state_groupby['Worked in County of Residence Estimate Aggregate (min)'].pct_change().dropna())*100
pct_change_in_state = (state_groupby['Worked in State of Residence Estimate Aggregate (min)'].pct_change().dropna())*100
pct_change_out_county = (state_groupby['Worked Outside County of Residence Estimate Aggregate (min)'].pct_change().dropna())*100
pct_change_out_state = (state_groupby['Worked Outside State of Residence Estimate Aggregate (min)'].pct_change().dropna())*100

county_state_change = pd.DataFrame({'State': states,
    'Change of Pop. Working in County of Residence': change_in_county,
    '% Change Working in County of Residence': pct_change_in_county,
    'Change of Pop. Working in State of Residence': change_in_state,
    '% Change Working in State of Residence': pct_change_in_state,
    'Change of Pop. Working Outside County of Residence': change_out_county,
    '% Change Working Outside County of Residence': pct_change_out_county,
    'Change of Pop. Working Outside State of Residence': change_out_state,
    '% Change Working Outside State of Residence': pct_change_out_state})
county_state_change = county_state_change.reset_index(drop=True)
county_state_change.style.format({'% Change Working in County of Residence': '{:,.2f}%', '% Change Working in State of Residence': '{:,.2f}%',
    '% Change Working Outside County of Residence': '{:,.2f}%', '% Change Working Outside State of Residence': '{:,.2f}%'})

,State,Change of Pop. Working in County of Residence,% Change Working in County of Residence,Change of Pop. Working in State of Residence,% Change Working in State of Residence,Change of Pop. Working Outside County of Residence,% Change Working Outside County of Residence,Change of Pop. Working Outside State of Residence,% Change Working Outside State of Residence
0,AL,3686770.000000,14.19%,6116075.000000,14.40%,2429305.000000,14.72%,561805.000000,17.30%
1,AK,313590.000000,6.31%,611815.000000,10.08%,298225.000000,27.21%,33065.000000,105.55%
2,AZ,18217950.000000,33.88%,19724125.000000,33.45%,1506175.000000,29.01%,330830.000000,20.99%
3,AR,1261035.000000,8.82%,2785640.000000,11.84%,1524605.000000,16.53%,-172580.000000,-10.40%
4,CA,91521270.000000,32.09%,132143470.000000,32.83%,40622200.000000,34.62%,572375.000000,19.95%
5,CO,8976610.000000,33.99%,17647975.000000,33.08%,8671360.000000,32.18%,194205.000000,13.86%
6,CT,2327485.000000,10.12%,2880740.000000,8.25%,553255.000000,4.64%,2374205.000000,42.56%
7,DE,1157770.000000,18.75%,1457295.000000,20.43%,299525.000000,31.26%,842405.000000,36.14%
8,DC,2107945.000000,38.98%,2107945.000000,38.98%,nan,nan%,911780.000000,31.53%
9,FL,46525885.000000,34.58%,68187320.000000,36.45%,21661440.000000,41.25%,1349135.000000,32.24%


In [32]:
# Min and Max percentages
max_in_county = county_state_change['% Change Working in County of Residence'].max()
min_in_county = county_state_change['% Change Working in County of Residence'].min()
max_in_state = county_state_change['% Change Working in State of Residence'].max()
min_in_state = county_state_change['% Change Working in State of Residence'].min()
max_out_county = county_state_change['% Change Working Outside County of Residence'].max()
min_out_county = county_state_change['% Change Working Outside County of Residence'].min()
max_out_state = county_state_change['% Change Working Outside State of Residence'].max()
min_out_state = county_state_change['% Change Working Outside State of Residence'].min()
# States corresponding to min and max
state_max_in_county = county_state_change[county_state_change['% Change Working in County of Residence'] == max_in_county]['State'].item()
state_min_in_county = county_state_change[county_state_change['% Change Working in County of Residence'] == min_in_county]['State'].item() 
state_max_in_state = county_state_change[county_state_change['% Change Working in State of Residence'] == max_in_state]['State'].item()
state_min_in_state = county_state_change[county_state_change['% Change Working in State of Residence'] == min_in_state]['State'].item()
state_max_out_county = county_state_change[county_state_change['% Change Working Outside County of Residence'] == max_out_county]['State'].item()
state_min_out_county = county_state_change[county_state_change['% Change Working Outside County of Residence'] == min_out_county]['State'].item()
state_max_out_state = county_state_change[county_state_change['% Change Working Outside State of Residence'] == max_out_state]['State'].item()
state_min_out_state = county_state_change[county_state_change['% Change Working Outside State of Residence'] == min_out_state]['State'].item()

print(f'Highest % change working in county of residence: {round(max_in_county,2)}%, {state_max_in_county}')
print(f'Lowest % change working in county of residence: {round(min_in_county,2)}%, {state_min_in_county}')
print(f'Highest % change working in state of residence: {round(max_in_state,2)}%, {state_max_in_state}')
print(f'Lowest % change working in state of residence: {round(min_in_state,2)}%, {state_min_in_state}')
print(f'Highest % change working ouside county of residence: {round(max_out_county,2)}%, {state_max_out_county}')
print(f'Lowest % change working outside county of residence: {round(min_out_county,2)}%, {state_min_out_county}')
print(f'Highest % change working outside state of residence: {round(max_out_state,2)}%, {state_max_out_state}')
print(f'Lowest % change working outside state of residence: {round(min_out_state,2)}%, {state_min_out_state}')

Highest % change working in county of residence: 39.27%, WA
Lowest % change working in county of residence: 1.71%, WV
Highest % change working in state of residence: 39.54%, WA
Lowest % change working in state of residence: -0.62%, WV
Highest % change working ouside county of residence: 45.19%, NV
Lowest % change working outside county of residence: -7.74%, WY
Highest % change working outside state of residence: 105.55%, AK
Lowest % change working outside state of residence: -10.4%, AR
